<p style="text-align: center;"><img src="https://upload.wikimedia.org/wikipedia/pt/1/1f/ITA_logo.png" alt="Resultado de imagem para logo ita" width="209" height="79" /></p>
<p style="text-align: center;"><span style="font-size: 14pt;">Instituto Tecnol&oacute;gico de Aeron&aacute;utica</span></p>
<p style="text-align: center;"><span style="font-size: 14pt;">P&oacute;s-Gradua&ccedil;&atilde;o em Engenharia Eletr&ocirc;nica e Computa&ccedil;&atilde;o</span></p>
<p style="text-align: center;"><span style="font-size: 14pt;"><strong>Orientador:</strong> Paulo Marcelo Tasinaffo</span></p>
<p style="text-align: center;"><span style="font-size: 14pt;"><strong>Co-orientador:</strong> Rodrigo Arnaldo Scarpel</span></p>
<p style="text-align: center;"><span style="font-size: 14pt;"><strong>Aluna:</strong> Bruna dos Santos Almeida</span></p>


## Import das Bibliotecas

In [1]:
import pandas as pd
import os
import time
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

from tensorflow import keras
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
#from keras.utils import to_categorical
from google.colab import drive
from sklearn.utils import class_weight

# Deep Learning Libraries
from keras.layers import Dense
from keras.models import Sequential
from keras.callbacks import EarlyStopping
from sklearn.utils import class_weight

drive.mount('/content/gdrive')

Mounted at /content/gdrive


## Import do Dataset

In [2]:
drive_path = "/content/gdrive/My Drive/Colab/Thesis/"
folder = "data/"
filename = "data_synthetic_"
folder_x_y = drive_path + folder + "train_test/"

In [3]:
data_five_features = list(range(1,37,3))
data_synthetic_list = []
count = 1
for item in data_five_features:
  dict_data = {}
  dict_data['key'] = item 

  print("Processing: "+str(count)+ " of "+str(len(data_five_features)))  
  print("Openning data train: "+str(item))
  count = count + 1
  data_train_path = os.path.join(folder_x_y, 'data_'+ str(item) + '_train.csv')
  data_train = pd.read_csv(data_train_path).drop("Unnamed: 0", axis=1)
  dict_data['train'] = data_train

  data_train_y = data_train['target']
  data_train_x = data_train.iloc[:,:-1]
  
  dict_data['train_y'] = keras.utils.to_categorical(data_train_y)
  dict_data['train_x'] = StandardScaler().fit_transform(data_train_x)
  
  #print("Openning data test: "+str(item))
  data_test_path = os.path.join(folder_x_y, 'data_'+ str(item) + '_test.csv')
  data_test = pd.read_csv(data_test_path).drop("Unnamed: 0", axis=1)
  dict_data['test'] = data_test

  dict_data['test_y'] = data_test['target']
  dict_data['test_x']  = data_test.iloc[:,:-1]
  
  #print("Salve ratio and early_stopping_monitor: "+str(item))
  non_default = len(data_train[data_train['target']==0])
  default = len(data_train[data_train['target']==1])
  ratio = float(default/(non_default+default))
  dict_data['ratio'] = ratio

  #n_cols = data_train_x.shape[1]
  class_weight = {0:ratio, 1:1-ratio}
  dict_data['class_weight'] = class_weight
  
  data_synthetic_list.append(dict_data)

Processing: 1 of 12
Openning data train: 1
Processing: 2 of 12
Openning data train: 4
Processing: 3 of 12
Openning data train: 7
Processing: 4 of 12
Openning data train: 10
Processing: 5 of 12
Openning data train: 13
Processing: 6 of 12
Openning data train: 16
Processing: 7 of 12
Openning data train: 19
Processing: 8 of 12
Openning data train: 22
Processing: 9 of 12
Openning data train: 25
Processing: 10 of 12
Openning data train: 28
Processing: 11 of 12
Openning data train: 31
Processing: 12 of 12
Openning data train: 34


# Artificial Neural Network Model

## Metrics

In [4]:
METRICS = [
      keras.metrics.BinaryAccuracy(name='accuracy'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='auc'),
      keras.metrics.TruePositives(name='tp'),
      keras.metrics.FalsePositives(name='fp'),
      keras.metrics.TrueNegatives(name='tn'),
      keras.metrics.FalseNegatives(name='fn')
]

# Models

In [5]:
def model_training(item, data_five_features,  data_train_x, data_train_y, data_test_x, data_test_y, 
                   class_weight, model, data_training, n_layer, 
                   nodes_from_1, nodes_from_2, nodes_from_3):
    model_total_iter = []
    for i in range(25):
      print("Run Iteration: "+str(i+1)+" in dataset "+str(item+1))
      model_iter = {}
      model_iter['key'] = i+1

      optimizer = tf.keras.optimizers.Adadelta(learning_rate=1)
      batch_size = 2000
      EPOCHS = 200
      early_stopping_monitor = EarlyStopping(patience=10)

      model.compile(optimizer=optimizer, 
                    loss='binary_crossentropy', 
                    metrics=METRICS)
      historic = model.fit(data_train_x, 
                            data_train_y,
                            epochs=EPOCHS, 
                            validation_split=0.25, 
                            callbacks = [early_stopping_monitor], 
                            class_weight=class_weight,
                            batch_size = batch_size)
      model_iter['historic'] = historic.history

      loss_test, accuracy_test, precision_test, recall_test, auc_test, tp_test, fp_test, tn_test, fn_test = model.evaluate(data_test_x, data_test_y, batch_size=128)

      new_row = {'layers': n_layer, 
                 'nodes_from_1': nodes_from_1, 
                 'nodes_from_2': nodes_from_2, 
                 'nodes_from_3': nodes_from_3, 
                 'iteration': i+1, 
                 'dataset': data_five_features[item],
                 'accuracy': historic.history['accuracy'], 
                 'val_accuracy': historic.history['val_accuracy'],
                 'loss': historic.history['loss'], 
                 'val_loss': historic.history['val_auc'],
                 'auc': historic.history['auc'], 
                 'val_auc': historic.history['val_auc'],
                 'precision': historic.history['precision'],
                 'val_precision':historic.history['val_precision'],
                 'recall': historic.history['recall'], 
                 'val_recall': historic.history['val_recall'],
                 'tp': historic.history['tp'], 
                 'val_tp': historic.history['val_tp'], 
                 'fp': historic.history['fp'], 
                 'val_fp': historic.history['val_fp'], 
                 'tn': historic.history['tn'], 
                 'val_tn': historic.history['val_tn'],
                 'fn': historic.history['fn'],
                 'val_fn': historic.history['val_fn'],
                 'loss_test': loss_test,
                 'accuracy_test': accuracy_test,
                 'precision_test': precision_test,
                 'recall_test': recall_test,
                 'auc_test': auc_test,
                 'tp_test': tp_test,
                 'fp_test': fp_test,
                 'tn_test': tn_test,
                 'fn_test': fn_test}

      data_training = data_training.append(new_row, ignore_index=True)
        
      model_total_iter.append(model_iter)
    return model_total_iter, data_training

## 5 Features

### 1 Layer

In [6]:
def features_5_model_1_hidden_layer():
    model = tf.keras.models.Sequential()
    model.add(tf.keras.Input(shape=(5,)))
    model.add(tf.keras.layers.Dense(3))
    model.add(tf.keras.layers.Dense(2, activation="softmax"))

    model.summary()
    return model

### 2 Layers

In [7]:
def features_5_model_2_hidden_layers():
    model = tf.keras.models.Sequential()
    model.add(tf.keras.Input(shape=(5,)))
    model.add(tf.keras.layers.Dense(4))
    model.add(tf.keras.layers.Dense(3))
    model.add(tf.keras.layers.Dense(2, activation="softmax"))

    model.summary()
    return model

### 3 Layers

In [8]:
def features_5_model_3_hidden_layers():
    model = tf.keras.models.Sequential()
    model.add(tf.keras.Input(shape=(5,)))
    model.add(tf.keras.layers.Dense(4))
    model.add(tf.keras.layers.Dense(3))
    model.add(tf.keras.layers.Dense(3))
    model.add(tf.keras.layers.Dense(2, activation="softmax"))

    model.summary()
    return model

### Run Models



In [9]:
def training(item, data_five_features, n_layer, nodes_from_1, nodes_from_2, nodes_from_3, model, 
             data_synthetic_list, data_training):

    data_train_x = data_synthetic_list[item]['train_x']
    data_train_y = data_synthetic_list[item]['train_y']

    data_test_x = data_synthetic_list[item]['test_x']
    data_test_y = data_synthetic_list[item]['test_y']

    data_test_y = pd.DataFrame(data=data_test_y)
    data_test_y.rename(columns = {'0':'target'}, inplace = True)
    data_test_y[0] = 0
    data_test_y[1] = 0
    data_test_y.loc[data_test_y['target'] == 0, 0] = 1
    data_test_y.loc[data_test_y['target'] == 1, 1] = 1
    data_test_y = data_test_y.drop('target', axis = 1)

    class_weight = data_synthetic_list[item]['class_weight']

    model_total_iter, data_training = model_training(item, data_five_features, data_train_x, data_train_y, 
                                   data_test_x, data_test_y, 
                                   class_weight, model, 
                                   data_training, n_layer, 
                                   nodes_from_1, nodes_from_2, nodes_from_3)
    
    data_synthetic_list[item]['training'] = model_total_iter

    return data_synthetic_list, data_training

# Test

In [10]:
file_name = "data_training_result_5.csv"
training_result_path = drive_path + folder + file_name
training_result_path

'/content/gdrive/My Drive/Colab/Thesis/data/data_training_result_5.csv'

In [11]:
for item in range(len(data_five_features)):
  try: 
    data_training = pd.read_csv(training_result_path)
  except:
    COLUMNS = ['layers', 'nodes_from_1', 'nodes_from_2', 'nodes_from_3',
               'dataset', 'iteration', 'accuracy', 'val_accuracy', 'loss', 
               'val_loss', 'auc', 'val_auc','precision', 'val_precision', 
               'recall', 'val_recall','tp', 'val_tp', 'fp', 'val_fp', 'tn', 
               'val_tn', 'fn', 'val_fn','loss_test', 'accuracy_test',
               'precision_test', 'recall_test', 'auc_test','tp_test',
               'fp_test','tn_test','fn_test']
    data_training = pd.DataFrame(columns = COLUMNS)

  n_layer = 1
  nodes_from_1 = 3
  nodes_from_2 = None
  nodes_from_3 = None
  model = features_5_model_1_hidden_layer()

  data_synthetic_list, data_training = training(item, data_five_features, n_layer, 
                                                nodes_from_1, nodes_from_2, 
                                                nodes_from_3, model, 
                                                data_synthetic_list, data_training)
  
  data_training.to_csv(training_result_path, index=False)

Streaming output truncated to the last 5000 lines.
Epoch 5/200
3/3 [==============================] - 0s 21ms/step - loss: 0.3382 - accuracy: 0.5716 - precision: 0.5716 - recall: 0.5716 - auc: 0.5812 - tp: 3215.0000 - fp: 2410.0000 - tn: 3215.0000 - fn: 2410.0000 - val_loss: 0.6835 - val_accuracy: 0.5733 - val_precision: 0.5733 - val_recall: 0.5733 - val_auc: 0.5914 - val_tp: 1075.0000 - val_fp: 800.0000 - val_tn: 1075.0000 - val_fn: 800.0000
Epoch 6/200
3/3 [==============================] - 0s 21ms/step - loss: 0.3382 - accuracy: 0.5712 - precision: 0.5712 - recall: 0.5712 - auc: 0.5809 - tp: 3213.0000 - fp: 2412.0000 - tn: 3213.0000 - fn: 2412.0000 - val_loss: 0.6835 - val_accuracy: 0.5733 - val_precision: 0.5733 - val_recall: 0.5733 - val_auc: 0.5914 - val_tp: 1075.0000 - val_fp: 800.0000 - val_tn: 1075.0000 - val_fn: 800.0000
Epoch 7/200
3/3 [==============================] - 0s 23ms/step - loss: 0.3382 - accuracy: 0.5724 - precision: 0.5724 - recall: 0.5724 - auc: 0.5816 - tp: 32

In [12]:
for item in range(len(data_five_features)):
  try: 
    data_training = pd.read_csv(training_result_path)
  except:
    COLUMNS = ['layers', 'nodes_from_1', 'nodes_from_2', 'nodes_from_3',
               'dataset', 'iteration', 'accuracy', 'val_accuracy', 'loss', 'val_loss',
               'auc', 'val_auc','precision', 'val_precision', 'recall', 
               'val_recall','tp', 'val_tp', 'fp', 'val_fp', 'tn', 'val_tn', 
               'loss_test', 'accuracy_test','precision_test', 'recall_test',
               'auc_test','tp_test','fp_test','tn_test','fn_test']
    data_training = pd.DataFrame(columns = COLUMNS)

  n_layer = 2
  nodes_from_1 = 4
  nodes_from_2 = 3
  nodes_from_3 = None
  model = features_5_model_2_hidden_layers()

  data_synthetic_list, data_training = training(item, data_five_features, n_layer, 
                                                nodes_from_1, nodes_from_2, 
                                                nodes_from_3, model, 
                                                data_synthetic_list, data_training)
  
  data_training.to_csv(training_result_path, index=False)

Streaming output truncated to the last 5000 lines.
Epoch 2/200
3/3 [==============================] - 0s 23ms/step - loss: 0.3382 - accuracy: 0.5733 - precision: 0.5733 - recall: 0.5733 - auc: 0.5818 - tp: 3225.0000 - fp: 2400.0000 - tn: 3225.0000 - fn: 2400.0000 - val_loss: 0.6835 - val_accuracy: 0.5744 - val_precision: 0.5744 - val_recall: 0.5744 - val_auc: 0.5913 - val_tp: 1077.0000 - val_fp: 798.0000 - val_tn: 1077.0000 - val_fn: 798.0000
Epoch 3/200
3/3 [==============================] - 0s 23ms/step - loss: 0.3382 - accuracy: 0.5710 - precision: 0.5710 - recall: 0.5710 - auc: 0.5800 - tp: 3212.0000 - fp: 2413.0000 - tn: 3212.0000 - fn: 2413.0000 - val_loss: 0.6834 - val_accuracy: 0.5712 - val_precision: 0.5712 - val_recall: 0.5712 - val_auc: 0.5912 - val_tp: 1071.0000 - val_fp: 804.0000 - val_tn: 1071.0000 - val_fn: 804.0000
Epoch 4/200
3/3 [==============================] - 0s 23ms/step - loss: 0.3382 - accuracy: 0.5717 - precision: 0.5717 - recall: 0.5717 - auc: 0.5815 - tp: 32

In [13]:
for item in range(len(data_five_features)):
  try: 
    data_training = pd.read_csv(training_result_path)
  except:
    COLUMNS = ['layers', 'nodes_from_1', 'nodes_from_2', 'nodes_from_3',
               'dataset', 'iteration', 'accuracy', 'val_accuracy', 'loss', 'val_loss',
               'auc', 'val_auc','precision', 'val_precision', 'recall', 
               'val_recall','tp', 'val_tp', 'fp', 'val_fp', 'tn', 'val_tn', 
               'loss_test', 'accuracy_test','precision_test', 'recall_test',
               'auc_test','tp_test','fp_test','tn_test','fn_test']
    data_training = pd.DataFrame(columns = COLUMNS)

  n_layer = 1
  nodes_from_1 = 4
  nodes_from_2 = 4
  nodes_from_3 = 3
  model = features_5_model_3_hidden_layers()

  data_synthetic_list, data_training = training(item, data_five_features, n_layer, 
                                                nodes_from_1, nodes_from_2, 
                                                nodes_from_3, model, 
                                                data_synthetic_list, data_training)
  
  data_training.to_csv(training_result_path, index=False)

Streaming output truncated to the last 5000 lines.
Epoch 19/200
3/3 [==============================] - 0s 23ms/step - loss: 0.4932 - accuracy: 0.4665 - precision: 0.4665 - recall: 0.4665 - auc: 0.4493 - tp: 2624.0000 - fp: 3001.0000 - tn: 2624.0000 - fn: 3001.0000 - val_loss: 0.9883 - val_accuracy: 0.4544 - val_precision: 0.4544 - val_recall: 0.4544 - val_auc: 0.4446 - val_tp: 852.0000 - val_fp: 1023.0000 - val_tn: 852.0000 - val_fn: 1023.0000
Epoch 20/200
3/3 [==============================] - 0s 23ms/step - loss: 0.4871 - accuracy: 0.4668 - precision: 0.4668 - recall: 0.4668 - auc: 0.4492 - tp: 2626.0000 - fp: 2999.0000 - tn: 2626.0000 - fn: 2999.0000 - val_loss: 0.9764 - val_accuracy: 0.4544 - val_precision: 0.4544 - val_recall: 0.4544 - val_auc: 0.4449 - val_tp: 852.0000 - val_fp: 1023.0000 - val_tn: 852.0000 - val_fn: 1023.0000
Epoch 21/200
3/3 [==============================] - 0s 23ms/step - loss: 0.4813 - accuracy: 0.4668 - precision: 0.4668 - recall: 0.4668 - auc: 0.4493 - tp:

In [14]:
def plot_history(history):
    
    # 'loss', 'accuracy', 'val_loss', 'val_accuracy'
    acc = history['accuracy']
    val_acc = history['val_accuracy']
    loss = history['loss']
    val_loss = history['val_loss']

    x = range(1, len(acc) + 1)

    plt.figure(figsize=(12, 5))
    
    plt.subplot(1, 2, 1)
    plt.plot(x, acc, 'b', label='Training acc')
    plt.plot(x, val_acc, 'r', label='Validation acc')
    plt.title('Training and validation accuracy')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(x, loss, 'b', label='Training loss')
    plt.plot(x, val_loss, 'r', label='Validation loss')
    plt.title('Training and validation loss')
    plt.legend()

In [15]:
#describe_models

In [16]:
#file_name = "data_training_result_5.csv"
#training_result_path = drive_path + folder + file_name
#describe_models.to_csv(training_result_path, index=False)